In [44]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.feature_extraction import DictVectorizer as DV


In [4]:
import xgboost as xgb


In [5]:

from xgboost import XGBRegressor

In [6]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin

data = pd.read_csv("train.csv")
data = data.drop_duplicates("description")
data = data.dropna(subset=['province', 'price'])
test_data = pd.read_csv("test.csv")

In [7]:
data.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,Portugal,This is a fine rich balanced wine. It has ripe...,Vila Santa Reserva,88.870874,20.0,Alentejano,NaN,NaN,NaN,NaN,NaN,PORTUGUESE RED,J. Portugal Ramos,32027
1,France,"A solid, chunky wine, with a structure that is...",NaN,88.041695,28.0,Bordeaux,Lalande de Pomerol,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Tour Grand Colombier,71079
2,France,"This is powerful and concentrated, with the hi...",NaN,94.085021,130.0,Bordeaux,Saint-Émilion,NaN,NaN,NaN,NaN,BORDEAUX-STYLE RED BLEND,Château Figeac,32440
3,US,"Rich, ripe and oaky, this Petite Sirah charms ...",Thompson Vineyard,89.869797,34.0,California,Santa Barbara County,Central Coast,NaN,NaN,Jaffurs 2010 Thompson Vineyard Petite Sirah (S...,PETITE SIRAH,Jaffurs,124405
4,US,This wine is a unique in the state blend and f...,McKinley Springs Vineyard,89.017651,24.0,Washington,Horse Heaven Hills,Columbia Valley,Sean P. Sullivan,@wawinereport,Syncline 2016 McKinley Springs Vineyard Rosé (...,ROSé,Syncline,33649


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123770 entries, 0 to 174999
Data columns (total 14 columns):
country                  123770 non-null object
description              123770 non-null object
designation              86887 non-null object
points                   123770 non-null float64
price                    123770 non-null float64
province                 123770 non-null object
region_1                 103639 non-null object
region_2                 52940 non-null object
taster_name              53628 non-null object
taster_twitter_handle    50884 non-null object
title                    65549 non-null object
variety                  123769 non-null object
winery                   123770 non-null object
id                       123770 non-null int64
dtypes: float64(2), int64(1), object(11)
memory usage: 14.2+ MB


In [9]:
data.describe()

,points,price,id
count,123770.000000,123770.000000,123770.000000
mean,88.159246,34.562939,70346.977636
std,3.154662,38.550039,41517.603113
min,79.636128,4.000000,1.000000
25%,85.987823,16.000000,34373.250000
50%,88.003258,25.000000,70230.500000
75%,90.117971,40.000000,105012.750000
max,100.220603,2500.000000,150927.000000


In [10]:
# data['winery'].value_counts(normalize=True)

In [11]:
categorical_features = ['country', 'description', 'designation',
                        'province','region_1','region_2','taster_name','taster_twitter_handle',
                        'title','variety','winery']
def fill_na(data, cols):
    for col in cols:
        data[col].fillna(value='nan', inplace=True)
fill_na(data, categorical_features)
fill_na(test_data, categorical_features)

In [12]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(data, data.price, test_size=0.2)


# variety_encoder = OneHotEncoder()
# variety_encoder.fit(np.array(data.variety).reshape(-1, 1))

# province_encoder = MultiLabelBinarizer()

# test_data = pd.read_csv("test.csv")

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, test_size=0.2)
# X_test = data[80000:]
# y_train = X_train.price.tolist()
# y_test = X_test.price.tolist()

variety_encoder = OneHotEncoder()
variety_encoder.fit(np.array(data.variety).reshape(-1, 1))

province_encoder = MultiLabelBinarizer()





x_train_province = []
for i in data.province:
    x_train_province.append(i.lower().split())

province_encoder.fit(x_train_province)

class ItemSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        if self.key == "province":
            x = []
            for i in data_dict[self.key]:
                x.append(i.lower().split())

            fitted = province_encoder.transform(x)
            return fitted
        elif self.key == 'variety':
            fitted = variety_encoder.transform(np.array(data_dict[self.key]).reshape(-1, 1))
            return fitted.toarray()
        elif self.key == 'points':
            return np.transpose(np.matrix(data_dict[self.key]))
        else:
            return data_dict[self.key]

def score(a, b):
    score = 0
    for i in range(len(a)):
        score += np.abs(a[i] - b[i]) / b[i]

    return score / len(a)

def regression(regressor_model):

    regressor = Pipeline([
        ('features', FeatureUnion([
            ('description', Pipeline([
                ('selector', ItemSelector(key='description')),
                ('vectorizer', TfidfVectorizer(stop_words = 'english'))
            ])),
            ('province', Pipeline([
                ('selector', ItemSelector(key='province'))
            ])),
            ('variety', Pipeline([
                ('selector', ItemSelector(key='variety'))
            ])),
            ('points', Pipeline([
                ('selector', ItemSelector(key='points'))
            ]))
        ])),
        ('regressor', regressor_model)
    ])

    #regressor = regressor.fit(X_train, data["price"][:80000])
    regressor = regressor.fit(X_train, y_train)
    print("Fitted")
    # test_X = data.description[80000:]
    #test_y = data["price"][80000:].tolist()
    predictions = regressor.predict(X_test)
    print("predicted")

    return {'test_score': score(predictions, y_test.tolist()), 'regressor':regressor}

# print("Ridge: " + str(regression(Ridge())))
# print("Gradient Boosting Regressor: " + str(regression(GradientBoostingRegressor())))
# arr = [8, 9, 10, 11]
# y = []
# for n in arr:
#     y.append(regression(KNeighborsRegressor(n_neighbors=n)))
#
# plt.plot(arr, y)
# plt.show()

# 11 and 1 give the lowest error
# 11 = 0.5165486593321018
# 5 = 0.5204515283809694
model = regression(KNeighborsRegressor(n_neighbors=10))


Fitted
predicted


In [14]:
print("Mean Percent Error:" + str(model['test_score']))

Mean Percent Error:0.3607509810308113


In [ ]:
return {'test_score': score(predictions, y_test.tolist()), 'regressor':regressor}


In [15]:
test_data[X_train.columns].sample(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
26084,US,"Composed of 100% Dijon clone Chardonnay, this ...",nan,88.240450,NaN,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Rex Hill 2006 Chardonnay (Oregon),CHARDONNAY,Rex Hill,26084
32214,Austria,"This light, crisp sparkling wine is just off d...",Blanc de Blancs,84.945917,NaN,Austria,nan,nan,Roger Voss,@vossroger,Dürnberg NV Blanc de Blancs Sparkling (Austria),SPARKLING BLEND,Dürnberg,32214


In [18]:
submission = model['regressor'].predict(test_data[data.columns])

/usr/lib/python3/dist-packages/sklearn/preprocessing/label.py:935: UserWarning: unknown class(es) ['barbara', 'carasului', 'cephalonie', 'county-condrieu', 'ilia', 'istra', 'kefallonian', 'lake', 'langenlois', 'markopoulo', 'mirditë', 'nan', 'rhein-mosel', 'skadar', 'slovenska', 'stirling', 'ticino', 'waitaki', 'zitsa'] will be ignored
  .format(sorted(unknown, key=str)))


ValueError: Found unknown categories ['TEMPRANILLO-MALBEC', 'SERCIAL', 'COLORINO', 'VERDIL', 'FRANCISA', 'HUXELREBE', 'VRANAC', 'TINTO VELASCO', 'TEROLDEGO ROTALIANO', 'BIANCOLELLA', 'MESEGUERA', 'COCOCCIOLA', 'VIURA-SAUVIGNON BLANC', 'AIDANI', 'MAVROTRAGANO', 'TSOLIKOURI', 'CABERNET SAUVIGNON-BARBERA', 'SYRAH-MALBEC', 'DIAMOND', 'MARIA GOMES-BICAL', 'PRUNELARD', 'ORANGETRAUBE', 'PIGNOLO', 'RIESLING-CHARDONNAY', 'KALLMET', 'RODITIS-MOSCHOFILERO', 'MELNIK', 'TăMâIOASă ROMâNEASCă', 'MALVAZIJA', 'NASCETTA', 'MOSCHOFILERO-CHARDONNAY', 'PIQUEPOUL BLANC', 'PARDINA', 'SHIRAZ-ROUSSANNE', 'VALVIN MUSCAT', 'FOLLE BLANCHE', 'TINTA AMARELA', 'THRAPSATHIRI', 'MUSCADINE', 'BLAUBURGER'] in column 0 during transform

In [17]:
test_data[X_train.columns]


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,id
0,US,"Sweet new oak stands out, giving this Cabernet...",3D,95.036469,NaN,California,St. Helena,Napa,nan,nan,Salvestrin 2007 3D Cabernet Sauvignon (St. Hel...,CABERNET SAUVIGNON,Salvestrin,0
1,France,Charming raspberry aromas intertwine with scen...,Heluicum,90.966405,NaN,Rhône Valley,Collines Rhôdaniennes,nan,nan,nan,nan,SYRAH,Les Vins de Vienne,1
2,US,"Ripe blackberry, leather and soy show on the n...",Estate,88.964358,NaN,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brave & Maiden 2013 Estate Cabernet Franc (San...,CABERNET FRANC,Brave & Maiden,2
3,US,White flowers and wild anise give this a crisp...,nan,89.960356,NaN,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Les Voleurs 2013 Chardonnay (Sonoma Coast),CHARDONNAY,Les Voleurs,3
4,US,Pinot Gris has been making inroads in Washingt...,nan,88.075501,NaN,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Chateau Ste. Michelle 2015 Pinot Gris (Columbi...,PINOT GRIS,Chateau Ste. Michelle,4
5,US,"An easy, gentle red wine, full-bodied and spic...",nan,86.118662,NaN,California,Russian River Valley,Sonoma,nan,nan,Bearboat 2008 Syrah (Russian River Valley),SYRAH,Bearboat,5
6,Argentina,Red berry fruit with touches of vanilla and sp...,Saurus Patagonia,86.054011,NaN,Other,Neuquén,nan,nan,nan,nan,PINOT NOIR,Familia Schroeder,6
7,US,"Tart, dry and simple, this Cab will happily wa...",nan,82.964697,NaN,California,California,California Other,nan,nan,Montevina 2006 Cabernet Sauvignon (California),CABERNET SAUVIGNON,Montevina,7
8,Italy,"A gorgeous and opulent nose with honey, vanill...",nan,88.950953,NaN,Northeastern Italy,Trentino,nan,nan,nan,nan,PINOT GRIGIO,Bollini,8
9,Italy,"Pressed violet, baked plum, baking spice and c...",nan,90.167277,NaN,Veneto,Amarone della Valpolicella Classico,nan,Kerin O’Keefe,@kerinokeefe,David Sterza 2011 Amarone della Valpolicella ...,RED BLEND,David Sterza,9


In [2]:
# cols_to_transform = [ "description ", "designation", "points", "province", "region_1", "region_2", 
#                 "taster_name","taster_twitter_handle","variety" ]
# df_with_dummies = pd.get_dummies( columns = cols_to_transform )


In [3]:
# vectorizer = DV( sparse = False )
# vec_x_cat_train = vectorizer.fit_transform( x_cat_train )
# vec_x_cat_test = vectorizer.transform( x_cat_test ) 

In [36]:
# sns.jointplot(y='price',x='winery',data=train_data)

In [41]:
# full_data = train_data.append(test_data)

In [ ]:
# drop_columns = ["description ", "designation", "points", "province", "region_1", "region_2", 
#                 "taster_name","taster_twitter_handle","variety"]
# full_data.drop(labels=drop_columns, axis=1, inplace=True)

In [ ]:
# full_data = pd.get_dummies(full_data, columns=["Sex"])
# full_data.fillna(value=0.0, inplace=True)

In [ ]:
# X_train = full_data.values[0:891]
# X_test = full_data.values[891:]

In [ ]:
# sudo apt-get install build-essential      # Ubuntu/Debian
# sudo yum groupinstall 'Development Tools' # CentOS/RHEL
# pip install xgboost

In [ ]:


# X_test = data[80000:]
# y_train = X_train.price.tolist()
# y_test = X_test.price.tolist()


In [ ]:
# from sklearn.preprocessing import scale
# from sklearn.model_selection import train_test_splitX = X.drop(columns='id') #id is our index and won't help our model
# X = scale(X) X_train, X_test, y_train, y_test = train_test_split(
#         X, y, train_size=0.75, test_size=0.25, random_state=42, shuffle=True)

In [19]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [23]:
categorical_features = ['country', 'description', 'designation',
                        'province','region_1','region_2','taster_name','taster_twitter_handle',
                        'title','variety','winery']
def fill_na(data, cols):
    for col in cols:
        data[col].fillna(value='nan', inplace=True)
fill_na(data, categorical_features)
fill_na(test_data, categorical_features)

In [24]:
dataset = loadtxt('train.csv', delimiter=",")

ValueError: could not convert string to float: 'country'